## Import required lib

In [2]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install SimpleITK

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install nibabel

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision.transforms import RandomHorizontalFlip, RandomRotation, ToTensor, Normalize
import nibabel as nib
from scipy.ndimage import zoom
import torch.nn as nn
from torchvision.models.video import r3d_18
import pandas as pd
import SimpleITK as sitk
import pickle

## Load the excel file with paths

In [6]:
final_df = pd.read_csv('/kaggle/input/csvfile/final_output.csv')
final_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2113 entries, 0 to 2112
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Subject        2113 non-null   object
 1   Age            2113 non-null   int64 
 2   Modality       2113 non-null   object
 3   Group          2113 non-null   object
 4   Image Data ID  2113 non-null   int64 
 5   path           2113 non-null   object
 6   filename       2113 non-null   object
dtypes: int64(2), object(5)
memory usage: 115.7+ KB


In [7]:
final_df

,Subject,Age,Modality,Group,Image Data ID,path,filename
0,128_S_0272,71,MRI,CN,68938,/kaggle/input/adnidata/ADNI/128_S_0272/MPR-R__...,ADNI_128_S_0272_MR_MPR-R__GradWarp__B1_Correct...
1,128_S_0272,73,MRI,CN,105494,/kaggle/input/adnidata/ADNI/128_S_0272/MPR-R__...,ADNI_128_S_0272_MR_MPR-R__GradWarp__B1_Correct...
2,128_S_0272,72,MRI,CN,69226,/kaggle/input/adnidata/ADNI/128_S_0272/MPR-R__...,ADNI_128_S_0272_MR_MPR-R__GradWarp__B1_Correct...
3,128_S_0272,71,PET,CN,57334,"/kaggle/input/adnidata/ADNI/128_S_0272/Coreg,_...","ADNI_128_S_0272_PT_Coreg,_Avg,_Standardized_Im..."
4,128_S_0272,74,PET,CN,149234,"/kaggle/input/adnidata/ADNI/128_S_0272/Coreg,_...","ADNI_128_S_0272_PT_Coreg,_Avg,_Standardized_Im..."
...,...,...,...,...,...,...,...
2108,029_S_1073,67,PET,MCI,1596802,"/kaggle/input/adnidata/ADNI/029_S_1073/Coreg,_...","ADNI_029_S_1073_PT_Coreg,_Avg,_Std_Img_and_Vox..."
2109,029_S_1073,67,PET,MCI,1596748,"/kaggle/input/adnidata/ADNI/029_S_1073/Coreg,_...","ADNI_029_S_1073_PT_Coreg,_Avg,_Std_Img_and_Vox..."
2110,029_S_1073,66,PET,MCI,1596851,"/kaggle/input/adnidata/ADNI/029_S_1073/Coreg,_...","ADNI_029_S_1073_PT_Coreg,_Avg,_Std_Img_and_Vox..."
2111,029_S_1073,68,PET,MCI,1596813,"/kaggle/input/adnidata/ADNI/029_S_1073/Coreg,_...","ADNI_029_S_1073_PT_Coreg,_Avg,_Std_Img_and_Vox..."


In [8]:
mri_df = final_df[final_df['Modality'] == 'MRI']
mri_paths = (mri_df['path'] +'/'+mri_df['filename']).to_numpy()
mri_labels = mri_df['Group'].to_numpy()

## Dataloader to create the dataset and split into test and train images

In [9]:
'''
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        #image = nib.load(image_path).get_fdata()
        #print(image.size())
        image = sitk.ReadImage(image_path)
        image = sitk.GetArrayFromImage(image)
        if self.transform is not None:
            image = self.transform(image)

        return image, label

# Define your image paths and labels as numpy arrays
image_paths = paths  # Replace with your own image paths
labels = labels  # Replace with your own labels
'''
#MRI
# Perform 70-30 train-test split
mri_train_image_paths, mri_test_image_paths, mri_train_labels, mri_test_labels = train_test_split(
    mri_paths, mri_labels, test_size=0.3, random_state=42)


In [10]:
print(mri_paths.shape)
print(mri_train_image_paths.shape)
print(mri_test_image_paths.shape)

(729,)
(510,)
(219,)


In [11]:
def pathstoarray(train_image_paths):
    train_images = []
    for image_path in train_image_paths:
        # Read image using OpenCV
        image = sitk.ReadImage(image_path)
        image = sitk.GetArrayFromImage(image)
        #print(image.shape)
        # Convert BGR image to RGB
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Append image to list
        image = np.resize(image, (60, 128, 128))
        train_images.append(image)
    return train_images

In [12]:
mri_train_images = np.array(pathstoarray(mri_train_image_paths))
mri_test_images = np.array(pathstoarray(mri_test_image_paths))

In [13]:
print(mri_train_images.shape)
print(mri_test_images.shape)


(510, 60, 128, 128)
(219, 60, 128, 128)


In [14]:
# mri_train_pkl = 'mri_train_array.pkl'
# with open(mri_train_pkl, 'wb') as f:
#     pickle.dump(mri_train_images, f)

# mri_test_pkl = 'mri_test_array.pkl'
# with open(mri_test_pkl, 'wb') as f:
#     pickle.dump(mri_test_images, f)
    
# pet_train_pkl = 'pet_train_array.pkl'
# with open(pet_train_pkl, 'wb') as f:
#     pickle.dump(pet_train_images, f)

# pet_test_pkl = 'pet_test_array.pkl'
# with open(pet_test_pkl, 'wb') as f:
#     pickle.dump(pet_test_images, f)

In [15]:
classes = np.array(['MCI', 'CN', 'AD'])
# Create a dictionary to map class names to their corresponding index values
class_to_idx = {class_name: idx for idx, class_name in enumerate(classes)}
# Use a list comprehension to convert the class names to index values
mri_labels_idx = np.array([class_to_idx[label] for label in mri_train_labels])
# Use the index values to create the one-hot encoded labels
mri_lb_oh = np.eye(len(classes))[mri_labels_idx]
mri_train_data_model = mri_train_images[:, np.newaxis, :,:,:]

In [16]:
print(mri_train_data_model.shape)
print(mri_lb_oh.shape)


(510, 1, 60, 128, 128)
(510, 3)


## RESNET model implementation

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, ZeroPadding3D,\
     Flatten, BatchNormalization, AveragePooling3D, Dense, Activation, Add  , Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


In [18]:
def res_identity(x, filters): 
  #renet block where dimension doesnot change.
  #The skip connection is just simple identity conncection
  #we will have 3 blocks and then input will be added

  x_skip = x # this will be used for addition with the residual block 
  f1, f2 = filters

  #first block 
  x = Conv3D(f1, kernel_size=(1,1, 1), strides=(1, 1,1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #second block # bottleneck (but size kept same with padding)
  x = Conv3D(f1, kernel_size=(3,3, 3), strides=(1,1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # third block activation used after adding the input
  x = Conv3D(f2, kernel_size=(1,1, 1), strides=(1,1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  # x = Activation(activations.relu)(x)

  # add the input 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

def res_conv(x, s, filters):
  '''
  here the input size changes''' 
  x_skip = x
  f1, f2 = filters

  # first block
  x = Conv3D(f1, kernel_size=(1,1, 1), strides=(s,s, s), padding='valid', kernel_regularizer=l2(0.001))(x)
  # when s = 2 then it is like downsizing the feature map
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # second block
  x = Conv3D(f1, kernel_size=(3,3, 3), strides=(1,1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #third block
  x = Conv3D(f2, kernel_size=(1,1, 1), strides=(1,1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)

  # shortcut 
  x_skip = Conv3D(f2, kernel_size=(1, 1,1), strides=(s,s, s), padding='valid', kernel_regularizer=l2(0.001))(x_skip)
  x_skip = BatchNormalization()(x_skip)

  # add 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

In [19]:
def resnet50():

  input_im = Input(shape=(1,60, 128, 128,)) # cifar 10 images size,
  x = ZeroPadding3D(padding=(1,1, 1))(input_im)

  # 1st stage
  # here we perform maxpooling, see the figure above

  x = Conv3D(64, kernel_size=(3, 3,3), strides=(1,1, 1))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)
  x = MaxPooling3D((1,1, 1), strides=(1, 1,1))(x)

  #2nd stage 
  # frm here on only conv block and identity block, no pooling

  x = res_conv(x, s=1, filters=(16, 64))
  x = res_identity(x, filters=(16, 64))
  x = res_identity(x, filters=(16, 64))

  # 3rd stage

  x = res_conv(x, s=2, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # 4th stage

  x = res_conv(x, s=2, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # 5th stage

  x = res_conv(x, s=2, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # ends with average pooling and dense connection

  x = AveragePooling3D((2,2, 2), padding='same')(x)

  x = Flatten()(x)
  x = Dense(3, activation='softmax', kernel_initializer='he_normal')(x) #multi-class

  # define the model 

  model = Model(inputs=input_im, outputs=x, name='Resnet50')

  return model

In [20]:
base_model_MRI = resnet50()

fc_layer1 = tf.keras.layers.Dense(512, activation='relu')(base_model_MRI.layers[-2].output)
fc_layer1 = tf.keras.layers.Dropout(0.8)(fc_layer1)
output_layer = tf.keras.layers.Dense(3, activation='sigmoid')(fc_layer1)
MRImodel = Model(inputs=base_model_MRI.input, outputs=output_layer)

In [21]:
MRImodel.compile('adam','categorical_crossentropy',metrics=['accuracy'])
MRImodel.fit(mri_train_data_model,mri_lb_oh,batch_size=16,epochs=50,verbose=1)

Epoch 1/50
32/32 [==============================] - 215s 6s/step - loss: 13.0166 - accuracy: 0.4451
Epoch 2/50
32/32 [==============================] - 207s 6s/step - loss: 7.5063 - accuracy: 0.4824
Epoch 3/50
32/32 [==============================] - 195s 6s/step - loss: 5.5396 - accuracy: 0.5824
Epoch 4/50
32/32 [==============================] - 194s 6s/step - loss: 5.1214 - accuracy: 0.5863
Epoch 5/50
32/32 [==============================] - 206s 6s/step - loss: 4.9616 - accuracy: 0.5863
Epoch 6/50
32/32 [==============================] - 196s 6s/step - loss: 4.8546 - accuracy: 0.5882
Epoch 7/50
32/32 [==============================] - 194s 6s/step - loss: 4.6832 - accuracy: 0.5608
Epoch 8/50
32/32 [==============================] - 193s 6s/step - loss: 4.6389 - accuracy: 0.6020
Epoch 9/50
32/32 [==============================] - 207s 6s/step - loss: 4.5790 - accuracy: 0.5980
Epoch 10/50
32/32 [==============================] - 197s 6s/step - loss: 4.4987 - accuracy: 0.5902
Epoch 11

In [26]:
mri_test_labels_idx = np.array([class_to_idx[label] for label in mri_test_labels])
mri_test_data_model = mri_test_images[:, np.newaxis, :,:,:]
print(mri_test_data_model.shape)

(219, 1, 60, 128, 128)


In [23]:

y_pred_probs = MRImodel.predict(mri_test_data_model)  # Predicted probabilities for each class
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels

# Compute the confusion matrix
confusion_mat = confusion_matrix(mri_test_labels_idx, y_pred)

# Compute evaluation metrics
accuracy = accuracy_score(mri_test_labels_idx, y_pred)
precision = precision_score(mri_test_labels_idx, y_pred, average='weighted')
recall = recall_score(mri_test_labels_idx, y_pred, average='weighted')
f1score = f1_score(mri_test_labels_idx, y_pred, average='weighted')

print("Confusion Matrix:\n", confusion_mat)
print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1score))

7/7 [==============================] - 23s 3s/step
Confusion Matrix:
 [[112   7   5]
 [  5  55   5]
 [  8   2  20]]
Accuracy: 0.8539
Precision: 0.8537
Recall: 0.8539
F1-score: 0.8538


In [24]:
print(y_pred_probs)

[[9.99843359e-01 1.73538968e-01 2.37106951e-03]
 [6.11381531e-01 4.62652326e-01 2.54765987e-01]
 [3.80504400e-01 7.21221387e-01 8.26075897e-02]
 [7.16028214e-01 4.22298402e-01 3.58121991e-01]
 [8.26208517e-02 2.29176670e-01 9.94731605e-01]
 [9.88315344e-01 4.32638824e-02 5.76346628e-02]
 [7.43740857e-01 5.30867577e-01 5.60298264e-01]
 [8.98263991e-01 2.65235841e-01 8.73614132e-01]
 [1.48764253e-02 9.82620120e-01 9.31040195e-05]
 [9.98353899e-01 2.97399551e-01 1.84107963e-02]
 [4.48048800e-01 5.82777441e-01 2.71328449e-01]
 [3.28819036e-01 4.05076236e-01 8.69230092e-01]
 [5.63032091e-01 5.43167114e-01 6.46910369e-01]
 [1.55537814e-01 7.99744546e-01 3.09493709e-02]
 [4.03074026e-01 5.35423577e-01 1.68986335e-01]
 [5.55610597e-01 5.97957551e-01 3.04589987e-01]
 [9.98933613e-01 3.08777213e-01 4.11371561e-03]
 [9.98680353e-01 1.43859327e-01 2.41113640e-02]
 [9.02232230e-01 7.61864245e-01 2.56969392e-01]
 [9.99755323e-01 1.66593388e-01 2.22019572e-03]
 [9.94844317e-01 1.37148708e-01 6.333905

In [25]:
MRImodel.save('MRImodel.h5')